# Readme
<br> 本代码为HuffMan编码解码的演示<br>
<br> 目录 <br>
<br> 1.字符数据处理<br>
<br> 2.HuffMan树构造<br>
<br> 3.数据编码<br>
<br> 4.数据解码<br>
<br> 5计算压缩比.<br>

# 1、字符处理

In [1]:
import docx

In [2]:
doc = docx.Document('大作业-数据压缩.docx')
pl_test = [paragraph.text for paragraph in doc.paragraphs] # doc的段落组成的列表
print(len(pl_test))

25


经过源文件和答应结果可知，段落0, 1, 2, 24是无用信息

In [3]:
for item in [0, 1, 2, 3, 24]: # 前四段和最后一段为冗余信息
    print('第%d段'%item, end='\t')
    if pl_test[item] is '':
        print('这一段内容为空')
        continue
    else:
        print(pl_test[item])

第0段	编写程序利用huffman、算术编码或者LZ编码其中之一对下述文件进行压缩和解压缩，给出压缩比。提交源程序和报告。
第1段	这一段内容为空
第2段	这一段内容为空
第3段	A public key infrastructure (PKI) supports the distribution, revocation and verification of public keys used for public key encryption, and enables linking of identities with public key certificates. A PKI enables users and systems to securely exchange data over the internet and verify the legitimacy of certificate-holding entities, such as webservers, other authenticated servers and individuals. The PKI enables users to authenticate digital certificate holders, as well as to mediate the process of certificate revocation, using cryptographic algorithms to secure the process.
第24段	这一段内容为空


In [4]:
# 正式数据段落列表
pl = [paragraph.text for paragraph in doc.paragraphs] # doc的段落组成的列表
# 弹出段落[0, 1, 2, 24],剩余长度21
pl.pop(0)
pl.pop(0)
pl.pop(0)
pl.pop(-1)
for index in range(len(pl)):
    print('第%d段'%index, pl[index])

第0段 A public key infrastructure (PKI) supports the distribution, revocation and verification of public keys used for public key encryption, and enables linking of identities with public key certificates. A PKI enables users and systems to securely exchange data over the internet and verify the legitimacy of certificate-holding entities, such as webservers, other authenticated servers and individuals. The PKI enables users to authenticate digital certificate holders, as well as to mediate the process of certificate revocation, using cryptographic algorithms to secure the process.
第1段 PKI certificates include a public key used for encryption and cryptographic authentication of data sent to or from the entity that was issued the certificate. Other information included in a PKI certificate includes identifying information about the certificate holder, about the PKI that issued the certificate, and other data including the certificate's creation date and validity period.
第2段 Without PKI, se

In [5]:
# 只统计docx的英文字符部分
sum_characters = 0
i = 1
for item in pl:
    length = len(item)
    sum_characters += length
    print('第%d段字符长度：%d'%(i, length))
    i += 1
print('总字符长度：%d'%sum_characters)

第1段字符长度：581
第2段字符长度：390
第3段字符长度：337
第4段字符长度：15
第5段字符长度：50
第6段字符长度：343
第7段字符长度：175
第8段字符长度：353
第9段字符长度：304
第10段字符长度：738
第11段字符长度：16
第12段字符长度：411
第13段字符长度：261
第14段字符长度：17
第15段字符长度：327
第16段字符长度：438
第17段字符长度：436
第18段字符长度：234
第19段字符长度：14
第20段字符长度：418
第21段字符长度：405
总字符长度：6263


In [6]:
# 创建字符字典
statics_table = [
                 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', \
                 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', \
                 'u', 'v', 'w', 'x', 'y', 'z', \
                 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', \
                 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', \
                 'U', 'V', 'W', 'X', 'Y', 'Z', \
                 '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', \
                 ' ', ',', '.', '(', ')', '-', '\"', '\'', ':', ';', '/','’'
                ]
# 初始化一个字符字典
statics_dic = {}
for ch in statics_table:
    statics_dic[ch] = 0

# 统计字符数据，写入字符字典
for item in pl:
# for item in s:
    for char in item:
        if char in statics_table:
            statics_dic[char] += 1
        # 考虑编码上可能存在的问题
        else:
            if char == u'\xa0': # Non-breaking space，'\xa0' 属于latin1（ISO/IEC_8859-1）中的扩展字符集字符，即空格，需要单独处理
                statics_dic[' '] += 1
            else: # 最后价差是否漏记字符
                print('未知',char)

In [7]:
print(statics_dic)
sum([i for i in statics_dic.values()]), sum_characters # 判断是否有漏记的字符

{'a': 372, 'b': 75, 'c': 246, 'd': 196, 'e': 632, 'f': 144, 'g': 66, 'h': 171, 'i': 484, 'j': 3, 'k': 24, 'l': 144, 'm': 65, 'n': 309, 'o': 308, 'p': 83, 'q': 4, 'r': 338, 's': 363, 't': 556, 'u': 161, 'v': 48, 'w': 50, 'x': 6, 'y': 86, 'z': 3, 'A': 40, 'B': 3, 'C': 31, 'D': 3, 'E': 2, 'F': 7, 'G': 3, 'H': 1, 'I': 33, 'J': 0, 'K': 26, 'L': 0, 'M': 1, 'N': 1, 'O': 3, 'P': 33, 'Q': 0, 'R': 3, 'S': 3, 'T': 8, 'U': 1, 'V': 0, 'W': 4, 'X': 2, 'Y': 0, 'Z': 0, '0': 8, '1': 3, '2': 5, '3': 0, '4': 0, '5': 5, '6': 0, '7': 2, '8': 0, '9': 2, ' ': 968, ',': 51, '.': 39, '(': 4, ')': 4, '-': 10, '"': 6, "'": 4, ':': 1, ';': 3, '/': 2, '’': 1}


(6263, 6263)

In [8]:
# 清洗数据，删除未出现的字符
statics_list = []
statics = list(statics_dic.items())

for item in statics:
    if item[1] != 0:
        statics_list.append(item)

print(len(statics), statics)
print()
print(len(statics_list), statics_list)

ver = 0
for item in statics_list:
    ver += item[1]
print(ver, sum_characters)

74 [('a', 372), ('b', 75), ('c', 246), ('d', 196), ('e', 632), ('f', 144), ('g', 66), ('h', 171), ('i', 484), ('j', 3), ('k', 24), ('l', 144), ('m', 65), ('n', 309), ('o', 308), ('p', 83), ('q', 4), ('r', 338), ('s', 363), ('t', 556), ('u', 161), ('v', 48), ('w', 50), ('x', 6), ('y', 86), ('z', 3), ('A', 40), ('B', 3), ('C', 31), ('D', 3), ('E', 2), ('F', 7), ('G', 3), ('H', 1), ('I', 33), ('J', 0), ('K', 26), ('L', 0), ('M', 1), ('N', 1), ('O', 3), ('P', 33), ('Q', 0), ('R', 3), ('S', 3), ('T', 8), ('U', 1), ('V', 0), ('W', 4), ('X', 2), ('Y', 0), ('Z', 0), ('0', 8), ('1', 3), ('2', 5), ('3', 0), ('4', 0), ('5', 5), ('6', 0), ('7', 2), ('8', 0), ('9', 2), (' ', 968), (',', 51), ('.', 39), ('(', 4), (')', 4), ('-', 10), ('"', 6), ("'", 4), (':', 1), (';', 3), ('/', 2), ('’', 1)]

64 [('a', 372), ('b', 75), ('c', 246), ('d', 196), ('e', 632), ('f', 144), ('g', 66), ('h', 171), ('i', 484), ('j', 3), ('k', 24), ('l', 144), ('m', 65), ('n', 309), ('o', 308), ('p', 83), ('q', 4), ('r', 338)

# 2、HuffMan树构造

## 编码步骤
<br> 1.符号按概率大到小排序，权重为概率； <br>
<br> 2.取概率最小的字符作为左结点，其次小的符号为右结点； <br>
<br> 3.将2中两个(最小)元素合并作为新的结点，权重为概率和； <br>
<br> 4.新结点和剩余结点重新排序； <br>
<br> 5.重复步骤(2),直到待排列的结点只剩一个（即为根结点）； <br>
<br> 6.最后产生的树状图就是Huffman树； <br>
<br> 7.左节点为0，右节点为1，从根节点到子节点的一条路径即是符号的码字； <br>

In [9]:
# 定义结点类
class Node(object):
    def __init__(self, name=None, value=None):
        self.name = name
        self.value = value
        self.parent = None
        self.left = None
        self.right = None

In [10]:
# 哈夫曼树类
class HuffmanTree(object):
    # 根据Huffman树的思想：以结点为基础，建立Huffman树
    def __init__(self, statics):
        # 叶子结点列表
        self.Leaf = [Node(part[0], part[1]) for part in statics]
        # huffman树结点列表，根结点为最后一个元素
        self.tree_node = []
        while len(self.Leaf)!=2:
            self.Leaf.sort(key=lambda node:node.value) # 从小到大排序
            tmp = Node(value=(self.Leaf[0].value + self.Leaf[1].value))
            # Leaf[0]为右结点，Leaf[1]为左结点
            # 绑定父子关系
            tmp.right = self.Leaf[0]
            tmp.left = self.Leaf[1]
            self.Leaf[0].parent = tmp
            self.Leaf[1].parent = tmp 
            # 压入huffman树
            self.tree_node.append(self.Leaf[0])
            self.tree_node.append(self.Leaf[1])
            # 将已经处理的叶子结点弹出结点列表
            self.Leaf.pop(0)
            self.Leaf.pop(0)
            self.Leaf.append(tmp)
            
        # 最后一个元素，按照叶子结点列表长度判断结束，不需要计算概率值和为1
        tmp = Node(name='root', value=(self.Leaf[0].value + self.Leaf[1].value))
        tmp.right = self.Leaf[0]
        tmp.left = self.Leaf[1]        
        self.Leaf[0].parent = tmp
        self.Leaf[1].parent = tmp
        self.tree_node.append(self.Leaf[0])
        self.tree_node.append(self.Leaf[1])
        self.tree_node.append(tmp)

## 一个测试的样例
![](example.png)

### 测试编码

In [11]:
statics_test = [('s7', 0.01), ('s6', 0.10), ('s5', 0.15), \
           ('s4', 0.17), ('s3', 0.18), ('s2', 0.19), \
           ('s1', 0.20)]
tree_test = HuffmanTree(statics_test)

In [12]:
for node in tree_test.tree_node:
    print(node.name, node.value)

s7 0.01
s6 0.1
None 0.11
s5 0.15
s4 0.17
s3 0.18
s2 0.19
s1 0.2
None 0.26
None 0.35
None 0.39
None 0.61
root 1.0


In [13]:
# 当前结点地址，当前结点name，当前结点value，父结点地址
for node in tree_test.tree_node:
    print(id(node), node.name, node.value, id(node.parent))

2211381154816 s7 0.01 2211381156720
2211381157392 s6 0.1 2211381156720
2211381156720 None 0.11 2211381494000
2211381156776 s5 0.15 2211381494000
2211381155320 s4 0.17 2211381494056
2211381493832 s3 0.18 2211381494056
2211381493888 s2 0.19 2211381494112
2211381493944 s1 0.2 2211381494112
2211381494000 None 0.26 2211381494168
2211381494056 None 0.35 2211381494168
2211381494112 None 0.39 2211381494224
2211381494168 None 0.61 2211381494224
2211381494224 root 1.0 1641454896


In [14]:
encode_table_test = []
for node in tree_test.tree_node:
    if node.name != None and node.name != 'root': # 选择出叶子结点
        ch = node.name
        leaf = node
        if leaf.parent: # 判断有无父结点
            parent = leaf.parent
        
        code = ''
        
        while True:
            if parent.left == leaf: # 左结点为1
                code += '0'
            else: # 右结点为0 
                code += '1'
            
            leaf = parent
            if leaf.parent: # 判断有无父结点
                parent = leaf.parent
            else:
                break
        
        # print(ch, code[::-1]) # 逆序
        encode_table_test.append([ch, code[::-1]])
    else:
        pass
encode_table_test

[['s7', '0111'],
 ['s6', '0110'],
 ['s5', '010'],
 ['s4', '001'],
 ['s3', '000'],
 ['s2', '11'],
 ['s1', '10']]

In [15]:
encoded_test = ''
for item in encode_table_test:
    encoded_test += str(item[1])
print(encoded_test)

011101100100010001110


### 测试解码

In [16]:
root = tree_test.tree_node[-1] # 最后一个结点是根结点
tmp = root
decode_test = ''

for bit in encoded_test:
    if tmp.name == None or tmp.name == 'root':
        if bit == '1': # 右结点
            tmp = tmp.right
        else: #左结点
            tmp = tmp.left
        
        # 当前结点的父结点
        if tmp.name != None and tmp.name != 'root': # 叶子结点
            decode_test += tmp.name
            tmp = root
print(encoded_test, decode_test)

011101100100010001110 s7s6s5s4s3s2s1


# 3、正式编码

In [17]:
# 创建huffman树
huffmantree = HuffmanTree(statics_list)

In [18]:
for node in huffmantree.tree_node:
    print(node.name, node.value, end='\t')

H 1	M 1	N 1	U 1	: 1	’ 1	E 2	X 2	7 2	9 2	/ 2	None 2	None 2	None 2	j 3	z 3	B 3	D 3	G 3	O 3	R 3	S 3	1 3	; 3	q 4	W 4	( 4	) 4	' 4	None 4	None 4	None 4	None 4	2 5	5 5	x 6	" 6	None 6	None 6	None 6	None 6	None 6	F 7	T 8	0 8	None 8	None 8	None 8	None 8	None 9	- 10	None 11	None 12	None 12	None 12	None 15	None 16	None 16	None 17	None 21	k 24	None 24	K 26	None 27	C 31	None 32	I 33	P 33	None 38	. 39	A 40	v 48	None 48	w 50	, 51	None 53	None 63	m 65	g 66	None 66	b 75	None 77	p 83	y 86	None 88	None 98	None 104	None 128	None 132	f 144	l 144	None 152	u 161	None 169	h 171	None 186	d 196	None 232	c 246	None 276	None 296	o 308	n 309	None 330	r 338	None 357	s 363	a 372	None 428	i 484	None 522	t 556	None 604	e 632	None 639	None 695	None 735	None 912	  968	None 1078	None 1236	None 1334	None 1647	None 2046	None 2570	None 3693	root 6263	

In [19]:
# 当前结点地址，当前结点name，当前结点value，父结点地址
for node in huffmantree.tree_node:
    print(id(node), '\t' ,node.name, '\t' ,node.value, '\t' ,id(node.parent))

2211381495288 	 H 	 1 	 2211381232864
2211381495232 	 M 	 1 	 2211381232864
2211381495176 	 N 	 1 	 2211381496968
2211381497752 	 U 	 1 	 2211381496968
2211381496352 	 : 	 1 	 2211381497640
2211381497024 	 ’ 	 1 	 2211381497640
2211381496072 	 E 	 2 	 2211381497696
2211381494784 	 X 	 2 	 2211381497696
2211381496240 	 7 	 2 	 2211381497584
2211381496408 	 9 	 2 	 2211381497584
2211381496912 	 / 	 2 	 2211381448776
2211381232864 	 None 	 2 	 2211381448776
2211381496968 	 None 	 2 	 2211381448832
2211381497640 	 None 	 2 	 2211381448832
2211381233424 	 j 	 3 	 2211381448888
2211381495400 	 z 	 3 	 2211381448888
2211381495904 	 B 	 3 	 2211381448944
2211381496128 	 D 	 3 	 2211381448944
2211381495960 	 G 	 3 	 2211381449000
2211381494840 	 O 	 3 	 2211381449000
2211381495568 	 R 	 3 	 2211381449056
2211381495680 	 S 	 3 	 2211381449056
2211381494728 	 1 	 3 	 2211381449112
2211381496856 	 ; 	 3 	 2211381449112
2211381494280 	 q 	 4 	 2211381449168
2211381494616 	 W 	 4 	 2211381449168
221

In [20]:
# 创建编码表
huffman_encode_table = []
huffman_encode_table_dic = {}

for node in huffmantree.tree_node:
    if node.name != None and node.name != 'root': # 选择出叶子结点
        ch = node.name
        leaf = node
        if leaf.name != 'root': # 判断有无父结点
            parent = leaf.parent
        code = ''
        
        while True:
            if parent.left == leaf: # 左结点为1
                code += '0'
            
            else: # 右结点为0 
                code += '1'
            
            leaf = parent
            if leaf.name != 'root': # 判断有无父结点
                parent = leaf.parent
            else:
                break
        
        huffman_encode_table.append([ch, code[::-1]])
    else:
        pass

# 列表转为字典
for item in huffman_encode_table:
    huffman_encode_table_dic[item[0]] = item[1]
print(huffman_encode_table_dic)

{'H': '111100111001', 'M': '111100111000', 'N': '111100110111', 'U': '111100110110', ':': '111100110101', '’': '111100110100', 'E': '010100100001', 'X': '010100100000', '7': '11110011111', '9': '11110011110', '/': '11110011101', 'j': '10000010101', 'z': '10000010100', 'B': '10000010011', 'D': '10000010010', 'G': '10000010001', 'O': '10000010000', 'R': '01010100111', 'S': '01010100110', '1': '01010100101', ';': '01010100100', 'q': '01010010101', 'W': '01010010100', '(': '01010010011', ')': '01010010010', "'": '01010010001', '2': '1111001100', '5': '1111001001', 'x': '1111001000', '"': '1000001011', 'F': '0101010001', 'T': '0101010000', '0': '0101001011', '-': '111100101', 'k': '10000011', 'K': '01010101', 'C': '01010011', 'I': '00010101', 'P': '00010100', '.': '1111000', 'A': '1000011', 'v': '1000010', 'w': '1000000', ',': '0101011', 'm': '0101000', 'g': '0001011', 'b': '111101', 'p': '101001', 'y': '101000', 'f': '000100', 'l': '11111', 'u': '10101', 'h': '10001', 'd': '01011', 'c': '0

In [21]:
# 开始编码，源文件的每一个段落进行编码
huffman_encoded = []
for para in pl:
    huffman_encoded_para = ''
    for char in para:
        if char == u'\xa0': # Non-breaking space，'\xa0' 属于latin1（ISO/IEC_8859-1）中的扩展字符集字符，即空格，需要单独处理
            huffman_encoded_para += huffman_encode_table_dic[' ']
        else:
            huffman_encoded_para += huffman_encode_table_dic[char]
    huffman_encoded.append(huffman_encoded_para)

# 编码结果
for para in huffman_encoded:
    print(para)

...


# 4、正式解码

## 解码步骤
<br> 解码只需要将经过编码的比特串不断从huffman树的根结点遍历即可；<br>
<br> 每到一次叶子结点，即解码出一个字符；<br>

In [22]:
# 最后一个结点是根结点
# 注：huffman树的结点是存储在列表中的，通过root寻找子结点遍历树
root = huffmantree.tree_node[-1]
huffman_decoded = []
for para in huffman_encoded:
    huffman_decoded_para = ''
    tmp = root
    for bit in para:
        if tmp.name == None or tmp.name == 'root':
            if bit == '1': # 右结点
                tmp = tmp.right
            else: #左结点
                tmp = tmp.left
            
            if tmp.name != None and tmp.name != 'root': # 叶子结点
                # print(tmp.name)
                huffman_decoded_para += tmp.name
                tmp = root
    huffman_decoded.append(huffman_decoded_para)

In [23]:
for para in huffman_decoded:
    print(para)

A public key infrastructure (PKI) supports the distribution, revocation and verification of public keys used for public key encryption, and enables linking of identities with public key certificates. A PKI enables users and systems to securely exchange data over the internet and verify the legitimacy of certificate-holding entities, such as webservers, other authenticated servers and individuals. The PKI enables users to authenticate digital certificate holders, as well as to mediate the process of certificate revocation, using cryptographic algorithms to secure the process.
PKI certificates include a public key used for encryption and cryptographic authentication of data sent to or from the entity that was issued the certificate. Other information included in a PKI certificate includes identifying information about the certificate holder, about the PKI that issued the certificate, and other data including the certificate's creation date and validity period.
Without PKI, sensitive info

# 5、计算压缩比

## 假设条件
### 不同的编程语言对于字符的存储内存分配机制不同。这里假设，一个字符分配1个字节，即8bit。huffman编码将字符编码为一个bit串，故能压缩编码长度。

In [24]:
# sum_characters：源文件需要编码的字符总数 6263
# huffman_encode_table_dic：编码表
# statics：字符出现频率统计


# 不压缩的数据大小，单位bit
none_encode = sum_characters * 8
# 采用huffman编码后的数据大小，单位bit
huff_encode = 0
for item in statics:
    ch = item[0]
    num = item[1]
    if num == 0:
        continue
    else:
        code = huffman_encode_table_dic[ch]

    huff_encode += num * len(code)

compression_ration = 1 - huff_encode / none_encode
    
print('不使用压缩算法，原字符占用内存：%d（bit）'%none_encode)
print('使用huffman编码算法，占用内存：%d（bit）'%huff_encode)
print('压缩比：%.4f%%'%(compression_ration*100))

不使用压缩算法，原字符占用内存：50104（bit）
使用huffman编码算法，占用内存：27603（bit）
压缩比：44.9086%
